In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
from ipywidgets import Output
out = Output()

In [2]:
!pip install --disable-pip-version-check -q pandas transformers seaborn tensorflow_hub elasticsearch elasticsearch-dsl torch annoy faiss-cpu
!pip install --disable-pip-version-check -qU scikit-learn nltk spacy
!pip -q install --disable-pip-version-check --no-warn-script-location --user tensorflow-text 
with out:
    !python -m spacy download en_core_web_sm
#UNCOMMENT IF RUNNING ON GPU BECAUSE TENSORFLOW TEXT DISABLES GPU ACCESS 
#!pip -q uninstall -y tensorflow

In [1]:
from utils import *

# Make sure Elasticsearch is alive

In [6]:
es = Elasticsearch(timeout=100)
es.ping()

True

In [13]:
es.indices.get_mapping(index="arg_index")

{'arg_index': {'mappings': {'properties': {'conclusion': {'type': 'text'},
    'context': {'properties': {'sourceId': {'type': 'keyword'}}},
    'stance': {'type': 'keyword'},
    'text': {'type': 'text', 'similarity': 'my_dirichlet'}}}}}

# Load the Topics, Judgments and Arguments into DataFrames

In [16]:
%%time
judgments = pd.read_csv('Data/tira-qrels', delim_whitespace=True, names=['topic','iteration','id','relevance'])
arguments = pd.read_pickle('Data/dataset.pkl')

tree = ET.parse('Data/topics-automatic-runs-task-1.xml')
root = tree.getroot()

topics = []
for child in root:
    d = {'topic':int(child[0].text), 'query':child[1].text}
    topics.append(d)
topics = pd.DataFrame(topics)
display(topics[:2])
display(judgments[:2])
display(arguments[:2])

,topic,query
0,1,Should Teachers Get Tenure?
1,2,Is Vaping with E-Cigarettes Safe?


,topic,iteration,id,relevance
0,9,0,5f1c7022-2019-04-18T15:36:48Z-00005-000,3
1,30,0,8e2fdd07-2019-04-18T15:40:20Z-00003-000,3


,text,stance,id,conclusion,context.sourceId,context.previousArgumentInSourceId,context.acquisitionTime,context.discussionTitle,context.sourceTitle,context.sourceUrl,context.nextArgumentInSourceId
0,My opponent forfeited every round. None of my ...,CON,c67482ba-2019-04-18T13:32:05Z-00000-000,Contraceptive Forms for High School Students,c67482ba-2019-04-18T13:32:05Z,,2019-04-18T13:32:05Z,Contraceptive Forms for High School Students,Debate Argument: Contraceptive Forms for High ...,https://www.debate.org/debates/Contraceptive-F...,c67482ba-2019-04-18T13:32:05Z-00001-000
1,How do you propose the school will fund your p...,CON,c67482ba-2019-04-18T13:32:05Z-00001-000,Contraceptive Forms for High School Students,c67482ba-2019-04-18T13:32:05Z,c67482ba-2019-04-18T13:32:05Z-00000-000,2019-04-18T13:32:05Z,Contraceptive Forms for High School Students,Debate Argument: Contraceptive Forms for High ...,https://www.debate.org/debates/Contraceptive-F...,c67482ba-2019-04-18T13:32:05Z-00002-000


CPU times: user 713 ms, sys: 409 ms, total: 1.12 s
Wall time: 1.12 s


# Load the semantic indices

In [5]:
%%time
current_dir = Path(".")
encoded_dir = current_dir / "Encoded"
tokenized_dir = current_dir / "Tokenized"
indices = {}
for file in encoded_dir.iterdir():
    annoy_index, arg_ids = make_or_load_annoy(embedding_posix=file, n_trees=5, b=2)
    pq_index, _ = make_or_load_pq(embedding_posix=file, b=2)
    indices[file.stem] = {"annoy_index":annoy_index, "pq_index":pq_index, "arg_ids":arg_ids}

annoy_d = indices['distilbert-base-uncased']['annoy_index']
pq_d = indices['distilbert-base-uncased']['pq_index']
ids_d = indices['distilbert-base-uncased']['arg_ids']

annoy_a = indices['autoencoded_distilbert-base-uncased_1024']['annoy_index']
pq_a = indices['autoencoded_distilbert-base-uncased_1024']['pq_index']
ids_a = indices['autoencoded_distilbert-base-uncased_1024']['arg_ids']

annoy_g = indices['UniversalSentenceEncoderEmbeddings']['annoy_index']
pq_g = indices['UniversalSentenceEncoderEmbeddings']['pq_index']
ids_g = indices['UniversalSentenceEncoderEmbeddings']['arg_ids']

Index already exists, now loading UniversalSentenceEncoderEmbeddings_angular_5
Now reading in argument ids
No tokenized pickle. Reading original dataset.
-------------------
-------ANNOY-------
-------------------
Index already exists, now loading UniversalSentenceEncoderEmbeddings_64_8
Now reading in argument ids
No tokenized pickle. Reading original dataset.
-------------------
-------FAISS-------
-------------------
Index already exists, now loading autoencoded_distilbert-base-uncased_1024_angular_5
Now reading in argument ids
Found tokenized pickle.
-------------------
-------ANNOY-------
-------------------
Index already exists, now loading autoencoded_distilbert-base-uncased_1024_64_8
Now reading in argument ids
Found tokenized pickle.
-------------------
-------FAISS-------
-------------------



Index already exists, now loading distilbert-base-uncased_angular_5_0_to_2
Now reading in argument ids
Found tokenized pickle.
-------------------
-------ANNOY-------
-------------------
Index already exists, now loading distilbert-base-uncased_64_8_0_to_2
Now reading in argument ids
Found tokenized pickle.
-------------------
-------FAISS-------
-------------------
CPU times: user 12.2 s, sys: 4.25 s, total: 16.4 s
Wall time: 17.4 s


# Load the embedding neural networks

In [8]:
#LOAD DISTILBERT
config = AutoConfig.from_pretrained('distilbert-base-uncased')
config.output_hidden_states=True
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
dbert = TFAutoModel.from_pretrained('distilbert-base-uncased', config=config)
#LOAD UNIVERSAL SENTENCE ENCODER
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
#LOAD AUTOENCODER
MODEL_TO_USE = 'distilbert-base-uncased'
model_version = "0001"
model_name = f"Encoder{MODEL_TO_USE}_1024"
folder_name = "Autoencoder_encoder"
model_path = Path('.') / folder_name / model_name / model_version
ae_e = tf.saved_model.load(str(model_path))

In [68]:
query = "Abortion is not good"
num_vectors_to_find = 10000

# Embed the query using every one of the embedders

In [51]:
%%time
#GET EMBEDDING FROM DISTILBERT
mod_input = tokenizer.encode_plus(query)
i = tf.constant(mod_input['input_ids'])[None, :]
m = tf.constant(mod_input['attention_mask'])[None, :]
output = dbert(i, attention_mask=m, training=False)
embedding = np.hstack([thing.numpy()[:,0,:] for thing in reversed(output[-1])])
d_embedding = embedding[:, 0*config.dim:2*config.dim]

#GET EMBEDDING FROM UNIVERSAL SENTENCE ENCODER
g_embedding = embed([query]).numpy()

#GET EMBEDDING FROM AUTOENCODER
a_embedding = embedding[:,:-config.dim]
a_embedding = ae_e(tf.constant(a_embedding)).numpy()

# Query FAISS and ANNOY for every embedding (6 different indices: 2 per embedding) and store the returned argument ids in a resultset corresponding to the index

In [52]:
%%time
result_indices = annoy_d.get_nns_by_vector(d_embedding.squeeze(), num_vectors_to_find, search_k=-1, include_distances=False)
set_d_annoy = set(ids_d['id'][result_indices].values)

result_indices = annoy_a.get_nns_by_vector(a_embedding.squeeze(), num_vectors_to_find, search_k=-1, include_distances=False)
set_a_annoy = set(ids_d['id'][result_indices].values)

result_indices = annoy_g.get_nns_by_vector(g_embedding.squeeze(), num_vectors_to_find, search_k=-1, include_distances=False)
set_g_annoy = set(ids_d['id'][result_indices].values)

_, result_indices = pq_d.search(d_embedding, num_vectors_to_find)  
result_indices = result_indices.squeeze()
set_d_pq = set(ids_d['id'][result_indices].values)

_, result_indices = pq_a.search(a_embedding, num_vectors_to_find)  
result_indices = result_indices.squeeze()
set_a_pq = set(ids_a['id'][result_indices].values)

_, result_indices = pq_g.search(g_embedding, num_vectors_to_find)  
result_indices = result_indices.squeeze()
set_g_pq = set(ids_g['id'][result_indices].values)

CPU times: user 2.93 s, sys: 0 ns, total: 2.93 s
Wall time: 429 ms


# Run query against the Elasticsearch index and store the resulting argument ids in a set

In [10]:
query = "Should Animals Be Used for Scientific or Commercial Testing?"

In [11]:
%%time
s = Search(using=es, index="arg_index")

s.query = Q("match", text=query)
s = s[:10000]

response = s.execute()

CPU times: user 117 ms, sys: 53.6 ms, total: 170 ms
Wall time: 2.62 s


In [13]:
elastic_set = set()
elastic_dic = {}
for hit in response:
    elastic_set.add(hit.meta.id)
    elastic_dic[hit.meta.id] = hit.meta.score

In [14]:
for hit in response[:20]:
    print(f"{hit.stance} ARGUMENT CONCLUSION: {hit.conclusion}")

PRO ARGUMENT CONCLUSION: Animals Should Be Used For Scientific or Commercial Testing.
PRO ARGUMENT CONCLUSION: Monkeys should be considered persons in the eyes of the law
CON ARGUMENT CONCLUSION: animal testing.
CON ARGUMENT CONCLUSION: Animals Should Be Used For Scientific or Commercial Testing.
CON ARGUMENT CONCLUSION: Experimenting on animals for science purposes shouldn't be banned
CON ARGUMENT CONCLUSION: Animal testing does more good than harm.
PRO ARGUMENT CONCLUSION: Animal testing should be banned
CON ARGUMENT CONCLUSION: Animal rights
CON ARGUMENT CONCLUSION: Should animal testing be banned
CON ARGUMENT CONCLUSION: The US federal Government should ban ALL testing that requires the use of animals.
PRO ARGUMENT CONCLUSION: Animal Testing should be allowed
CON ARGUMENT CONCLUSION: Animal Testing should be allowed
PRO ARGUMENT CONCLUSION: Monkeys should be considered persons in the eyes of the law
CON ARGUMENT CONCLUSION: animal expirimentation
CON ARGUMENT CONCLUSION: Animal Tes

# Expand the original query using transformer-based language models, both causal and not

## Use a Cloze (Masked, BERT-like) Language Model to generate a list of keywords relevant to the original query and run those against the Elasticsearch index

In [44]:
tokenizer_mlm = AutoTokenizer.from_pretrained('bert-large-uncased')
model_mlm = BertForMaskedLM.from_pretrained('bert-large-uncased')

In [45]:
%%time
keywords = expand_mlm(model=model_mlm, tokenizer=tokenizer_mlm, query="Should Animals Be Used for Scientific or Commercial Testing?", k=3)
print(keywords)

['infection', 'toxicity', 'development', 'human', 'accidental', 'blood', 'poverty', 'technology', 'research', 'infancy', 'animal', 'pregnancy', 'health', 'humans', 'stress', 'death', 'medicine', 'testing', 'cost', 'injury', 'disease', 'science', 'birth', 'fear', 'animals']
CPU times: user 38.8 s, sys: 692 ms, total: 39.5 s
Wall time: 3.29 s


In [47]:
%%time
s = Search(using=es, index="arguments_index")
s.query = Q("match", text=' '.join(keywords))
s = s[:10000]

response = s.execute()
mlm_set = set()
mlm_dic = {}
for hit in response:
    mlm_set.add(hit.meta.id)
    mlm_dic[hit.meta.id] =hit.meta.score

CPU times: user 379 ms, sys: 8.05 ms, total: 387 ms
Wall time: 1.03 s


In [48]:
for hit in response[:20]:
    print(f"{hit.stance} ARGUMENT CONCLUSION: {hit.conclusion}")

PRO ARGUMENT CONCLUSION: The US federal Government should ban ALL testing that requires the use of animals.
CON ARGUMENT CONCLUSION: Medical testing on animals does more good than harm.
CON ARGUMENT CONCLUSION: animal testing.
CON ARGUMENT CONCLUSION: Animal Testing does more harm than good
PRO ARGUMENT CONCLUSION: Stem cell research on all types of stem cells should be completely legal
PRO ARGUMENT CONCLUSION: Animals should be tested on
PRO ARGUMENT CONCLUSION: Animals Should Be Used For Scientific or Commercial Testing.
CON ARGUMENT CONCLUSION: Should animals be used for research (pro:yes con:no)
PRO ARGUMENT CONCLUSION: Animal Testing
PRO ARGUMENT CONCLUSION: Ethics Is More Important Than Science
CON ARGUMENT CONCLUSION: Animal Testing
PRO ARGUMENT CONCLUSION: People for Ethical Treatment of Animals
PRO ARGUMENT CONCLUSION: Animal Testing
CON ARGUMENT CONCLUSION: Animal Testing
CON ARGUMENT CONCLUSION: Animal testing for medical research
PRO ARGUMENT CONCLUSION: Animal Testing
CON 

## Use a causal transformer language model (GPT2-like) to hallucinate new queries and run those against the Elasticsearch index

In [2]:
LM_MODEL_TO_USE = "gpt2"
tokenizer_lm = AutoTokenizer.from_pretrained(LM_MODEL_TO_USE)
model_lm = AutoModelWithLMHead.from_pretrained(LM_MODEL_TO_USE)

In [3]:
%%time
greedy, sampled = expand_lm(model=model_lm, tokenizer=tokenizer_lm, query="Should Animals Be Used for Scientific or Commercial Testing?")

 
Hallucinated 1 for query 1:  it's not a science.What is your favourite movie of all times, any more than most people would love an ice cream sandwich that they have on their plate when the day before has arrived and eaten at 3pm in front Ofelia St.? Well I guess some kids are just looking up to movies from such as The Muppets which had its timepiece set upon by John Carpenter while being
 
 
Hallucinated 2 for query 1:  that's what they are. So the fact we have animals on our farms is an important point of emphasis in animal welfare policy and scientific research so there needs to be evidence, but this will not only help with a good comparison as opposed by saying if something does exist then it must necessarily work within human beings' rights law."On July 1st at 4:00 pm Eastern time (12am
 
 
Hallucinated 3 for query 1:  of how much effort it takes to make sure animals have the proper diet, nutrition and training. But this also helps humans better understand their own needs so that

In [39]:
%%time
s = Search(using=es, index="arguments_index")
s.query = Q('bool', should=[Q("match", text=query) for query in chain(*[greedy, sampled])], minimum_should_match=int(len(list(chain(*[greedy, sampled])))/2))
s = s[:10000]
response = s.execute()
lm_set = set()
lm_dic = {}
for hit in response:
    lm_set.add(hit.meta.id)
    lm_dic[hit.meta.id] =hit.meta.score

CPU times: user 340 ms, sys: 24.4 ms, total: 365 ms
Wall time: 34.2 s


In [40]:
for hit in response[:20]:
    print(f"{hit.stance} ARGUMENT CONCLUSION: {hit.conclusion}")

CON ARGUMENT CONCLUSION: animal testing.
PRO ARGUMENT CONCLUSION: Animals Should Be Used For Scientific or Commercial Testing.
CON ARGUMENT CONCLUSION: Should animals be used for research (pro:yes con:no)
PRO ARGUMENT CONCLUSION: Animal Testing
CON ARGUMENT CONCLUSION: Resolved: An alternative for animal testing should be found.
CON ARGUMENT CONCLUSION: Medical testing on animals does more good than harm.
CON ARGUMENT CONCLUSION: Animal Testing
CON ARGUMENT CONCLUSION: Should animals be used for research (pro:yes con:no)
PRO ARGUMENT CONCLUSION: Resolved: An alternative for animal testing should be found.
PRO ARGUMENT CONCLUSION: We should have lab rats in labatories
CON ARGUMENT CONCLUSION: Animal Testing does more harm than good
PRO ARGUMENT CONCLUSION: Animal Testing
CON ARGUMENT CONCLUSION: Animal Testing
CON ARGUMENT CONCLUSION: Resolved: An alternative for animal testing should be found.
CON ARGUMENT CONCLUSION: Animal Testing does more harm than good
CON ARGUMENT CONCLUSION: ani

In [54]:
#Interestingly, (lm_dic.keys() & mlm_dic.keys()) returns a set! Same as doing lm_set.intersect(mlm_set)
len(lm_dic.keys() & mlm_dic.keys()) == len(lm_set.intersection(mlm_set))

True

## Use a Bag-of-Words (arg_bow) enhanced GPT2 model (Uber's PPLM: Plug-and-Play Language Model) to hallucinate queries and run those against the Elasticsearch index

### The Bag of Words used attempts to steer the hallucinated text into sounding like an argument:

In [55]:
with open('arg_bow') as f:
    bow = f.read().splitlines()
print(bow)

['accept', 'according', 'accordingly', 'affirm', 'agree', 'argue', 'argument', 'argumentation', 'assert', 'assumption', 'attack', 'attest', 'backing', 'basis', 'because', 'belief', 'believe', 'bias', 'biased', 'challenge', 'cite', 'claim', 'clear', 'con', 'concede', 'conclude', 'conclusion', 'concur', 'consequence', 'consequently', 'considering', 'context', 'controversial', 'convince', 'corroborate', 'convincing', 'corroboration', 'credibility', 'credible', 'criteria', 'criterion', 'debatable', 'debate', 'deduce', 'definition', 'determine', 'disagree', 'disprove', 'ergo', 'evidence', 'example', 'facts', 'fallacy', 'fallible', 'faulty', 'general', 'hence', 'hypothetical', 'imply', 'inconsistent', 'infer', 'irrelevant', 'justify', 'knowledge', 'logical', 'naturally', 'objectively ', 'opinion', 'perspective', 'persuade', 'persuasive', 'point', 'position', 'precisely', 'premise', 'pro', 'probable', 'proof', 'prove', 'rational', 'reason', 'rebuttal', 'reiterate', 'relevant', 'rhetoric', 'rh

In [2]:
config = AutoConfig.from_pretrained("gpt2")
config.output_hidden_states = True
tokenizer_pplm = AutoTokenizer.from_pretrained("gpt2")
model_pplm = AutoModelWithLMHead.from_pretrained("gpt2", config=config)
model_pplm.eval()    
for param in model_pplm.parameters():
    param.requires_grad = False

In [5]:
%%time
pplm_hallucinated = expand_pplm(model=model_pplm, tokenizer=tokenizer_pplm, query="Should Animals Be Used for Scientific or Commercial Testing?", length=40, num_iterations=4)

, but it's not a scientific question. It is an ethical one and I don't want to be the first person who says that since animals are used in research they should never have been tested on
, but it's not a scientific question because the animals are already in their natural habitat and they're just trying to survive on what we call "natural" food sources like grasses that have been grown
 Because of the scientific and commercial implications.<|endoftext|>The following is a list compiled by me, based on my own experience with this project:

 (1) I have been working in an office environment where
, it's a good question! It should be asked because the scientific community has been very clear that animals are not used in any way to test their knowledge of science and therefore they don't have an
, but it's a good question. It would be interesting to see if there are any scientific studies that show the benefits of using animals in research and clinical trials as well as other forms thereof (e

In [7]:
%%time
s = Search(using=es, index="arguments_index")
s.query = Q('bool', should=[Q("match", text=query) for query in pplm_hallucinated], minimum_should_match=int(len(pplm_hallucinated)/2))
response = s.execute()
pplm_set = set()
pplm_dic = {}
for hit in response:
    pplm_set.add(hit.meta.id)
    pplm_dic[hit.meta.id] =hit.meta.score

CPU times: user 9.86 ms, sys: 4.11 ms, total: 14 ms
Wall time: 3.54 s


In [8]:
for hit in response[:20]:
    print(f"{hit.stance} ARGUMENT CONCLUSION: {hit.conclusion}")

CON ARGUMENT CONCLUSION: Animal Testing
PRO ARGUMENT CONCLUSION: Animals Should Be Used For Scientific or Commercial Testing.
PRO ARGUMENT CONCLUSION: We should have lab rats in labatories
CON ARGUMENT CONCLUSION: Should animals be used for research (pro:yes con:no)
CON ARGUMENT CONCLUSION: Medical Research does more harm than good
CON ARGUMENT CONCLUSION: animal testing.
PRO ARGUMENT CONCLUSION: Animal Testing Should be Banned
CON ARGUMENT CONCLUSION: Should animal testing be banned
CON ARGUMENT CONCLUSION: Veganism and Animal Rights
CON ARGUMENT CONCLUSION: Animal Testing


# Get union and k-wise intersection of the resultsets of argument ids

The k-wise intersection takes all possible combinations of k out of kmax resultsets and produces the intersection. The result is the union of all these intersections

In [94]:
k=5
union = set.union(*[set_d_annoy, set_a_annoy, set_g_annoy, set_d_pq, set_a_pq, set_g_pq])

from itertools import combinations
k_intersection = set()
for set_list in combinations([set_d_annoy, set_a_annoy, set_g_annoy, set_d_pq, set_a_pq, set_g_pq, elastic_set], r=k):
    k_intersection.update(set.intersection(*set_list))

,text,stance,id,conclusion,context.sourceId,context.previousArgumentInSourceId,context.acquisitionTime,context.discussionTitle,context.sourceTitle,context.sourceUrl,context.nextArgumentInSourceId
544,I am against this statement that abortion is m...,CON,cb76cede-2019-04-18T11:58:57Z-00001-000,Abortion is Murder,cb76cede-2019-04-18T11:58:57Z,cb76cede-2019-04-18T11:58:57Z-00000-000,2019-04-18T11:58:57Z,Abortion is Murder,Debate Issue: Abortion is Murder | Debate.org,https://www.debate.org/debates/Abortion-is-Mur...,cb76cede-2019-04-18T11:58:57Z-00002-000
8762,Abortion is inmoral/inhumane - 1st round accep...,CON,242e2ea6-2019-04-18T11:32:58Z-00001-000,Abortion is inmoral and inhumane,242e2ea6-2019-04-18T11:32:58Z,242e2ea6-2019-04-18T11:32:58Z-00000-000,2019-04-18T11:32:58Z,Abortion is inmoral and inhumane,Debate Argument: Abortion is inmoral and inhum...,https://www.debate.org/debates/Abortion-is-inm...,
8895,knowledge is the mechanism,CON,6737184-2019-04-18T15:34:09Z-00002-000,There is no mechanism that permits free will t...,6737184-2019-04-18T15:34:09Z,6737184-2019-04-18T15:34:09Z-00001-000,2019-04-18T15:34:09Z,There is no mechanism that permits free will t...,Debate Issue: There is no mechanism that permi...,https://www.debate.org/debates/There-is-no-mec...,6737184-2019-04-18T15:34:09Z-00003-000
10062,any right actions is moral,PRO,c4ac5612-2019-04-18T12:54:27Z-00005-000,morality=right+intent,c4ac5612-2019-04-18T12:54:27Z,c4ac5612-2019-04-18T12:54:27Z-00004-000,2019-04-18T12:54:27Z,morality=right+intent,Debate Issue: morality=right+intent | Debate.org,https://www.debate.org/debates/morality-right-...,
11199,A fetus is still a human being therefore shoul...,CON,b1852c44-2019-04-18T18:27:05Z-00005-000,Abortion,b1852c44-2019-04-18T18:27:05Z,b1852c44-2019-04-18T18:27:05Z-00004-000,2019-04-18T18:27:05Z,Abortion,Debate Topic: Abortion | Debate.org,https://www.debate.org/debates/Abortion/146/,b1852c44-2019-04-18T18:27:05Z-00000-000
...,...,...,...,...,...,...,...,...,...,...,...
387347,Common prostitution is not a biblical conflict,PRO,87d0ccd3-2019-04-17T11:47:45Z-00062-000,Prostitution,87d0ccd3-2019-04-17T11:47:45Z,87d0ccd3-2019-04-17T11:47:45Z-00001-000,2019-04-17T11:47:45Z,Prostitution,Debate: Prostitution - Debatepedia,http://www.debatepedia.org/en/index.php/Debate...,87d0ccd3-2019-04-17T11:47:45Z-00047-000
387348,Prostitutes are vulnerable because they lack l...,PRO,87d0ccd3-2019-04-17T11:47:45Z-00047-000,Prostitution,87d0ccd3-2019-04-17T11:47:45Z,87d0ccd3-2019-04-17T11:47:45Z-00062-000,2019-04-17T11:47:45Z,Prostitution,Debate: Prostitution - Debatepedia,http://www.debatepedia.org/en/index.php/Debate...,87d0ccd3-2019-04-17T11:47:45Z-00032-000
387355,Governments should not set moral sexual standa...,PRO,87d0ccd3-2019-04-17T11:47:45Z-00063-000,Prostitution,87d0ccd3-2019-04-17T11:47:45Z,87d0ccd3-2019-04-17T11:47:45Z-00002-000,2019-04-17T11:47:45Z,Prostitution,Debate: Prostitution - Debatepedia,http://www.debatepedia.org/en/index.php/Debate...,87d0ccd3-2019-04-17T11:47:45Z-00048-000
387360,Legalizing drugs is the least bad option,PRO,a7c47a5c-2019-04-17T11:47:49Z-00026-000,Legalization of drugs,a7c47a5c-2019-04-17T11:47:49Z,,2019-04-17T11:47:49Z,Legalization of drugs,Debate: Legalization of drugs - Debatepedia,http://www.debatepedia.org/en/index.php/Debate...,a7c47a5c-2019-04-17T11:47:49Z-00011-000


# Rerank a list of arguments using Distilbert and/or the Wide and Deep Network

## Load the Distilbert ranker

In [103]:
model_version = "0002"
model_name = f"distilbert-base-uncased_for_sequence_classification"
folder_name = "Ranker"
model_path = Path('.') / folder_name / model_name / model_version
db_rank = tf.saved_model.load(str(model_path))

## Rerank the intersection set with Distilbert

In [136]:
%%time
db_reranked = []
for i, arg in return_args(k_intersection).iterrows():
    tokenized = tokenizer.encode_plus(query, arg['text'], max_length=tokenizer.max_len, pad_to_max_length=True)
    i= tf.constant(tokenized['input_ids'], dtype=tf.int64)[None, :]
    m= tf.constant(tokenized['attention_mask'], dtype=tf.int64)[None, :]
    r,b = (np.array(db_rank([i,m], training=False)).squeeze())
    db_reranked.append({"id": arg['id'], 'relevance':r, 'binary':b})

CPU times: user 16min 49s, sys: 2min 34s, total: 19min 23s
Wall time: 1min 8s


In [138]:
db_reranked = pd.DataFrame(db_reranked)
db_reranked

,id,relevance,binary
0,cb76cede-2019-04-18T11:58:57Z-00001-000,-2.154010,0.047416
1,242e2ea6-2019-04-18T11:32:58Z-00001-000,-0.921782,0.259330
2,6737184-2019-04-18T15:34:09Z-00002-000,0.625956,0.808973
3,c4ac5612-2019-04-18T12:54:27Z-00005-000,-0.334137,0.478341
4,b1852c44-2019-04-18T18:27:05Z-00005-000,0.966623,0.895657
...,...,...,...
528,87d0ccd3-2019-04-17T11:47:45Z-00062-000,-0.417742,0.463349
529,87d0ccd3-2019-04-17T11:47:45Z-00047-000,3.092331,0.992629
530,87d0ccd3-2019-04-17T11:47:45Z-00063-000,-0.895341,0.358341
531,a7c47a5c-2019-04-17T11:47:49Z-00026-000,3.113705,0.993508


## Load the Wide and Deep ranker

In [101]:
model_version = "0001"
model_name = f"Wide_and_deep"
folder_name = "Ranker"
model_path = Path('.') / folder_name / model_name / model_version
wnd_rank = tf.saved_model.load(str(model_path))

## Load the QA version of the Universal Sentence Encoder

In [102]:
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-qa/3')
query_embedder = module.signatures['question_encoder']
arg_embedder = module.signatures['response_encoder']

INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-qa/3'.
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder-qa/3: 170.04MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder-qa/3: 330.04MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder-qa/3: 480.04MB
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder-qa/3, Total size: 588.94MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-qa/3'.


In [180]:
%%time
use_reranked = []
for i, arg in return_args(k_intersection).iterrows():
    q = query_embedder(tf.constant([query]))['outputs'].numpy()
    a = arg_embedder(input=tf.constant([arg['text']]), context=tf.constant([arg['conclusion']]))['outputs'].numpy()
    dp = np.dot(q.squeeze(),a.squeeze())
    deep_input = np.hstack([q,a])
    s = Search(using=es, index='arg_index')
    s.query = Q("match", text=query) & Q("term", _id=arg['id'])
    response = s.execute()
    length = len(arg['text'])
    ds = 0
    if not response.hits.total.value == 0:
        ds = response[0].meta.score
    wide_input = np.hstack([dp, ds, length])
    r,b,aux = np.array(wnd_rank([tf.constant(wide_input, dtype=tf.float32)[None,:], tf.constant(deep_input, dtype=tf.float32)], training=False)).squeeze()
    use_reranked.append({"id": arg['id'], 'relevance':r, 'binary':b, 'aux':aux})

CPU times: user 3min 50s, sys: 47.8 s, total: 4min 37s
Wall time: 56.1 s


In [182]:
use_reranked = pd.DataFrame(use_reranked)
use_reranked

,id,relevance,binary,aux
0,cb76cede-2019-04-18T11:58:57Z-00001-000,-1.502114,0.600468,0.026989
1,242e2ea6-2019-04-18T11:32:58Z-00001-000,1.445340,0.404336,0.947017
2,6737184-2019-04-18T15:34:09Z-00002-000,0.346199,0.231686,0.716269
3,c4ac5612-2019-04-18T12:54:27Z-00005-000,-1.694745,0.500830,0.067210
4,b1852c44-2019-04-18T18:27:05Z-00005-000,1.025825,0.435745,0.967622
...,...,...,...,...
528,87d0ccd3-2019-04-17T11:47:45Z-00062-000,-0.184481,0.398180,0.889010
529,87d0ccd3-2019-04-17T11:47:45Z-00047-000,1.824704,0.541169,0.991493
530,87d0ccd3-2019-04-17T11:47:45Z-00063-000,2.003159,0.601674,0.990718
531,a7c47a5c-2019-04-17T11:47:49Z-00026-000,2.114252,0.567398,0.970235


# Combine the two rankings using the scaled scores

In [259]:
scaler = MinMaxScaler()
all_scores = np.hstack([scaler.fit_transform(use_reranked[['relevance', 'binary', 'aux']]), scaler.fit_transform(db_reranked[['relevance', 'binary']])])
final_ranks = pd.DataFrame({'arg_ids':use_reranked['id'], 'mean':pd.Series(np.mean(all_scores, axis=1)), 'median':pd.Series(np.median(all_scores, axis=1))})
final_ranks

,arg_ids,mean,median
0,cb76cede-2019-04-18T11:58:57Z-00001-000,0.126117,0.027077
1,242e2ea6-2019-04-18T11:32:58Z-00001-000,0.469289,0.248732
2,6737184-2019-04-18T15:34:09Z-00002-000,0.505933,0.536868
3,c4ac5612-2019-04-18T12:54:27Z-00005-000,0.258220,0.350304
4,b1852c44-2019-04-18T18:27:05Z-00005-000,0.667831,0.604580
...,...,...,...
528,87d0ccd3-2019-04-17T11:47:45Z-00062-000,0.451419,0.359522
529,87d0ccd3-2019-04-17T11:47:45Z-00047-000,0.831818,0.995368
530,87d0ccd3-2019-04-17T11:47:45Z-00063-000,0.573685,0.481558
531,a7c47a5c-2019-04-17T11:47:49Z-00026-000,0.847082,0.974026


In [260]:
final_ranks.sort_values(by='mean')

,arg_ids,mean,median
8,53fcdf2e-2019-04-18T11:15:19Z-00002-000,0.086013,0.063137
11,b18614c4-2019-04-18T15:36:50Z-00003-000,0.115511,0.059338
115,1fce0d2d-2019-04-15T20:24:15Z-00004-000,0.115831,0.066849
113,1fce0d2d-2019-04-15T20:24:15Z-00007-000,0.121149,0.121743
189,d66b8937-2019-04-15T20:22:46Z-00005-000,0.123123,0.054322
...,...,...,...
531,a7c47a5c-2019-04-17T11:47:49Z-00026-000,0.847082,0.974026
478,c4e8d546-2019-04-17T11:47:34Z-00101-000,0.850499,0.928091
486,c4e8d546-2019-04-17T11:47:34Z-00111-000,0.856114,0.963035
429,d8150fb5-2019-04-17T11:47:48Z-00039-000,0.863684,0.921121


In [258]:
final_ranks.sort_values(by='median')

,arg_ids,mean,median
36,47ca0144-2019-04-18T15:38:01Z-00002-000,0.154978,0.026419
0,cb76cede-2019-04-18T11:58:57Z-00001-000,0.126117,0.027077
89,c5dcd8ac-2019-04-18T18:48:18Z-00005-000,0.292210,0.033648
37,d952cd48-2019-04-18T12:45:38Z-00001-000,0.147782,0.035299
374,b67fc3fb-2019-04-17T11:47:41Z-00079-000,0.126919,0.045069
...,...,...,...
68,70fa99e5-2019-04-18T11:07:40Z-00001-000,0.791173,0.976830
277,c297a6b3-2019-04-17T11:47:23Z-00034-000,0.802204,0.978417
206,bb88f107-2019-04-15T20:22:39Z-00001-000,0.811195,0.979332
88,cbde2446-2019-04-18T20:01:18Z-00002-000,0.759873,0.981733


In [268]:
sum(use_reranked['relevance'] < -0.5)

130